# Lab 1 Part 1 -- RSA basics and getting started
This lab will take you through the basics of encryption and decryption of RSA

There will be quite a few things we will need to talk about, so get prepared and get excited!

This, and subsequent labs, will try to work from simpler concepts to more complex.  One problem we find with working with RSA is you need to have a strong understanding of multiple concepts, some of the concepts will be foreign to you unless you were a double Math / CompSci major.

This is good news / bad news, you may not be familiar with all the concepts, but you should be familiar with some.  If you feel unfamiliar with none, then with a small amount of catch up you should be right on par.

This lab will cover some ancillary ideas which will lead to everything coming together in the end.

Of note, this lab is intended to cover the implementation of RSA, so you can see it work in practice.  Our hope is that we will start with small easy to handle numbers, and you can see that the concepts aren't inherently difficult.  We will add increasingly larger numbers and increasingly professional implementations, and you should see that nothing really changed except for the sizes of numbers.  But since we are using a computer the size of the number is immaterial to us.  
{We use SageMath since the size of the number is hugely material to the computer! And we can easily overrun memory if we aren't careful}

***NOTE***  
*The material and functions contained within will be required for the midterm.*

### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted  
2. _Encoding_ the string so it can be encrypted  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. _Decoding_ encoding to get plaintext  

### How does RSA Work?
RSA uses some "simple" mathematical operations for encryption and decryption.
The first operation is exponentiation.
The second operation is reduction by modulus.

When we say exponentiation we are quite simply taking a number, $m$, and raising it to a power, $e$, so we get $m^e$

The nomenclature:  
$m$ is the message to be encrypted.  (In this class this will need to be the encoded plaintext, but more on that later.)  
$e$ is an encryption exponent.  This is also called the public key, since $e$ will be made publicly available.  
$d$ is a decryption exponent.  This is also called the private key, since no one except the keyholder should be able to decrypt a transmisison.  

In [ ]:
################
# DATA DATA DATA
################


# for now we will use these values for RSA.
# I am including quite a few values here, in case you want to explore
# the only ones we will use now are Ea, Da, and Na

# Alice data
Pa = 2**107 - 1
Qa = 2**607 - 1
Na = Pa*Qa                    # Alice modulus
phi_Na = (Pa - 1)*(Qa - 1)
Ea = 101                      # Alice encryption exponent
Da = inverse_mod(Ea, phi_Na)  # Alice decryption exponent

# Bob data
Pb = 2**521 - 1
Qb = 2**127 - 1 
Nb = Pb*Qb                    # Bob modulus
phi_Nb = (Pb - 1)*(Qb - 1)
Eb = 173                      # Bob encryption exponent
Db = inverse_mod(Eb, phi_Nb)  # Bob decryption exponent

# a standard RSA key will have two parts, a public and private part
# here the public key will be the encryption exponent, and the modulus
# <Ea, Na> is the public key of Alice
# the private key Alice doesn't tell anyone, is <Da, Na>

### RSA ToDo List
Alice:  
1. **Declaring a _String_ to be encrypted**
2. _Encoding_ the string so it can be encrypted  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. _Decoding_ encoding to get plaintext  

Since we need to perform math to encrypt and decrypt in RSA, we need to use numbers.  
Say I want to transmit a secret message like, `"Have a nice day."`  
Well RSA says we simply raise this message to an encryption exponent.  Does it make sense to do this then: 

$\big("Have\ a\ nice\ day."\big)^e$  

Hopefully not.  We need to convert this string `"Have a nice day."` into an integer value of some sort so using an exponent makes sense..  This converstion is called encoding.  
Encoding occurs all the time in computers, ASCII is a common encoding, although for our purposes straight ASCII can be problematic.  
In fact, there are little problems which arise all the time when trying to find a good encoding scheme.  
We will be playing with `openssl` later, and it uses a common encoding called `-base64`.

For now we will make it easy for you.  I have devised a set of proprietary functions called `encode_message` and `decode_message`.

### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted
2. **_Encoding_ the string so it can be encrypted**  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. _Decoding_ encoding to get plaintext 

In [ ]:
####################################################
# The following functions are written and functional 
# They are for your use
####################################################


# ENCODE FUNCTION
# function to take ascii plaintext and convert to decimal value message
def encode_message(message):
    encoded = message.encode()
    encoded_message = int.from_bytes(encoded, byteorder='little')
    return encoded_message

# DECODE FUNCTION
# function to take decimal value message and convert to ascii plaintext
def decode_message(message):
    m = int(message)
    length = math.ceil(m.bit_length() / 8)
    decoded_message = m.to_bytes(length, byteorder='little').decode()
    return decoded_message

In [ ]:
################
# DEMO DEMO DEMO
################


# Let's check to see if the encoding works for us

# make a string
message = "HeLl@ w0r1d!"

# it's always important to know what type of object
# we are working with

# let's see what type of variable message is...
# should be a string type
print("The type of object before encoding: \n\t" + str(type(message)))

In [ ]:
# You can see in the output that 'message' is of the 
# class 'str' which means string.  Woot.

# now let's encode and turn it into an integer
encoded_message = encode_message(message)

# check type
# print message and encoded_message together
print("The type of object after encoding: \n\t" + str(type(encoded_message)))
print()

# print message
print("The original message: \n\t" + message)
print()
print("The encoded message: \n\t" + str(encoded_message))
print()

# print decoded message just to see it work...
# note the capitalization now... part of my basic encoding scheme
print("The decoded message: \n\t" + decode_message(encoded_message))

Just to talk about the encoding a bit.  
Consider a simplified code which uses the two digit ascii value.\
You can see this if you look up an ascii table.  

For instance, the letter `H` has an ascii value of 72. The `E` has a value of 69, `L` a value of 76, etc...  

So encoding `HELLO` would be `7269767679`\
In this way we can convert back and forth between this integer and a readable string.

*TECHNICAL NOTE*. 
It is important to check your encoding against your data types.  This is supposed to be a simplified example, but mistakes are still possible.  So, for our purposes you should always be using a _string_ as your message.  For our encoding it is just fine to use integers and numbers all you want.  So you can encode the string "1232221" just fine.  But you won't be able to encode an integer valued at 1232221. You can mix and match numbers in your string also, for instance you can say "HELLO, THE SECRET MESSAGE IS 1232221, AND I AM A SPY!". This is fine.  Enjoy!

In [ ]:
################
# YOUR WORK HERE
################


############
# Question 1
############

# encode these three messages
message_one = 'I LOVE MACARONI AND CHEESE'
message_two = 'I LOVE SAMOSAS AND LASSI'
message_three = 'I LOVE KIM CHI JI GAE'







# take those three messages and print them an their 
# corresponding encoding in a professional manner









### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted
2. _Encoding_ the string so it can be encrypted  
3. **_Encrypting_ the encoding to get ciphertext**  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. _Decoding_ encoding to get plaintext 

Let's do some encrypting.  After all of this setup, the actual encryption is quite straightforward.  

RSA is executed by taking the encoded message, an integer, raising it to an exponent, and finally reducing by a modulus.  

If we let $m$ be the encoded message, then raising it to an exponent is $m^e$. 

Finally, reducing by a modulus can be accomplished, in general by the modulus operator %, so we can say `m^e%N`.  

There is a problem with python and most other languages.\
There is rarely support built in for operations on large numbers.\
By large we mean values larger than the operating system can handle natively.\
If you're using a 64-bit operating system, which is most of us,\
then anything over the size of $2^64$ is threatening to be inaccurately processed.\
You will learn more about integer overflow errors and attacks in future MICS courses!  

Think about RSA, we commonly use RSA-2048... which uses values which are 2048 bits long... far exceeding the 64-bit size.\
Sagemath is specifically written to solve this issue and address other cryptological computational issues.

If you use python you could quickly and simply write the RSA encryption as `m^e%N`.\
In sagemath we use a function called `power_mod(m, e, N)`

In [ ]:
################
# DEMO DEMO DEMO
################


# Alice wants to send a message to Bob
# she's encocded it already as encoded_message above
# since she is sending a message to Bob, she will use Bob's 
# public encryption exponent, Eb, and his public modulus Nb

ciphertext = power_mod(encoded_message, Eb, Nb)

# what does the ciphertext look like?
# well we expect it to be a large integer
print("ciphertext: \n\t" + str(ciphertext))

### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted
2. _Encoding_ the string so it can be encrypted  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. **_Decrypting_ ciphertext to get encoding**  
2. _Decoding_ encoding to get plaintext 

Now Alice has sent an encrypted message to Bob.  This message can be intercepted and no one will be able to read it unless they have access to Bob's private key, Db, and Bob's modulus Nb.

So let's decrypt it!

RSA decryption is as simple as taking the ciphertext and raising it to the decryption exponent and simplifying by the modulus.

We use power_mod() again.  Super easy.  Just remember that this will return the encoded message, so that will be the final step.

In [ ]:
################
# YOUR WORK HERE
################

# say we receive the following ciphertext from Alice
ciphertext = 795922927188376388653614112175281831943327080979057468959583174351003302368934997582479902442311322447974967933691752065791135393698905728510519778941997861806227476464976904776206351788387574543

# decrypt ciphertext received from Alice
# we use bob's private key, Db, and modulus Nb
decrypted_message = power_mod(ciphertext, Db, Nb)

# what does the decrypted (but not decoded) message look like?
print("decrypted message: \n\t" + str(decrypted_message))
print()

############
# Question 2
############

# decode the message








### RSA ToDo List
Alice:  
1. Declaring a _String_ to be encrypted
2. _Encoding_ the string so it can be encrypted  
3. _Encrypting_ the encoding to get ciphertext  

Bob:  
1. _Decrypting_ ciphertext to get encoding  
2. **_Decoding_ encoding to get plaintext** 

In [ ]:
# let's decode this message and find out what Alice has to say!
plaintext_message = decode_message(decrypted_message)
print(plaintext_message)

Great, now it's time to try yourself!

In [ ]:
############
# Question 3
############

# Step 1 -- create a message you want to send to Bob
# call it my_message
# print my_message

my_message = ''
print(my_message)

In [ ]:
############
# Question 4
############

# Step 2 -- encode the string, save the encoding as my_encoded
# print my_encoded

# Step 3 -- encrypt the encoding, save it as my_ciphertext
# print my_ciphertext

In [ ]:
############
# Question 5
############

# Step 4 -- now act like bob and verify that he can read your message
# decrypt the message and save it as bob_encoded
# print bob_encoded

# Step 5 -- finally act like bob and decode the message
# print decoded message